# 실험을 돌리기 위한 코드
학습을 진행하고, 테스트 데이터에 대한 inference를 통해 submission파일을 만드는 튜토리얼

In [ ]:
from ultralytics import YOLO
import os

try:
    os.rmdir("./runs/")
except:
    pass

# Load a model
model = YOLO("yolov8x.yaml")  # build a new model from scratch
model = YOLO("yolov8x.pt")  # load a pretrained model (recommended for training)
#model.model.args['conf'] = 0.7
model.model.args['device'] = 0
model.model.args['optimizer'] = 'Adam'
# Use the model
#model.train(data="./dacon_data.yaml", epochs=200,batch=8)  # train the model
model.train(data="./dacon_data.yaml",seed=42, epochs=400,batch=8,imgsz=(1024,1024),optimizer='Adam',lr0=1e-3,augment=True,val=True,cache=True,workers=16,patience=5)  # train the model
metrics = model.val()  # evaluate model performance on the validation set


                   from  n    params  module                                       arguments                     
  0                  -1  1      2320  ultralytics.nn.modules.conv.Conv             [3, 80, 3, 2]                 
  1                  -1  1    115520  ultralytics.nn.modules.conv.Conv             [80, 160, 3, 2]               
  2                  -1  3    436800  ultralytics.nn.modules.block.C2f             [160, 160, 3, True]           
  3                  -1  1    461440  ultralytics.nn.modules.conv.Conv             [160, 320, 3, 2]              
  4                  -1  6   3281920  ultralytics.nn.modules.block.C2f             [320, 320, 6, True]           
  5                  -1  1   1844480  ultralytics.nn.modules.conv.Conv             [320, 640, 3, 2]              
  6                  -1  6  13117440  ultralytics.nn.modules.block.C2f             [640, 640, 6, True]           
  7                  -1  1   3687680  ultralytics.nn.modules.conv.Conv             [640

In [3]:
from ultralytics import YOLO
model = YOLO("./runs/detect/train123/weights/best.pt")
results = model.predict(source="./test", imgsz=(1024, 1024), iou=0.2, conf=0.5, save_conf=True, save=False, save_txt=True,
                  exist_ok=True, device=0, augment=True, verbose=False)
#results = model(source="./test")


    WARNING ⚠️ stream/video/webcam/dir predict source will accumulate results in RAM unless `stream=True` is passed,
    causing potential out-of-memory errors for large sources or long-running streams/videos.

    Usage:
        results = model(source=..., stream=True)  # generator of Results objects
        for r in results:
            boxes = r.boxes  # Boxes object for bbox outputs
            masks = r.masks  # Masks object for segment masks outputs
            probs = r.probs  # Class probabilities for classification outputs

Results saved to runs/detect/predict
3400 labels saved to runs/detect/predict/labels


In [4]:
"""
res.path.split("/")[-1]: 사진 이름
res.boxes.cls: class_id
res.boxes.conf: confidence score
res.boxes.xyxy: lower left, upper right
"""
for_submit = []
for i in range(len(results)):
    n = len(results[i].boxes.cls)
    for j in range(n):
        x_min, y_min, x_max, y_max = map(int,results[i].boxes.xyxy[j])
        #x_min, y_min, x_max, y_max = map(float,results[i].boxes.xyxy[j])
        for_submit.append([results[i].path.split("/")[-1], int(results[i].boxes.cls[j]), float(results[i].boxes.conf[j]), x_min, y_max, x_max, y_max, x_max, y_min, x_min, y_min])

In [5]:
import pandas as pd
submission = pd.DataFrame(for_submit)
col = pd.read_csv("./sample_submission.csv").columns
submission.columns = col
#submission = submission[submission['confidence']>=0.7]
submission.head(5)

,file_name,class_id,confidence,point1_x,point1_y,point2_x,point2_y,point3_x,point3_y,point4_x,point4_y
0,064442001.png,7,0.965588,1138,493,1428,493,1428,181,1138,181
1,064507368.png,16,0.975452,495,419,762,419,762,124,495,124
2,065131036.png,22,0.932766,1109,300,1358,300,1358,55,1109,55
3,065131036.png,33,0.819513,1109,295,1356,295,1356,56,1109,56
4,065147868.png,16,0.976393,727,491,1011,491,1011,187,727,187


In [6]:
import os
sub_dir = "./submissions/"
file_num = str(len(os.listdir(sub_dir)))
submission.to_csv(sub_dir + "submission" + file_num + ".csv",index=False)